In [18]:
import os
import pandas as pd
import openai
import faiss
import numpy as np


In [4]:
openai.api_key = os.getenv('OPENAI_KEY')


In [9]:
df = pd.read_csv('large_table_column_dataset_with_pii.csv')


In [11]:
df.head()

,Column Name,Table Name,Sample Data,Column Description,Is PII
0,user_id,users,"[100, 101, 102, 103, 104]",Unique identifier for each user,Yes
1,first_name,users,"['Hank', 'Bob', 'David', 'Grace', 'Grace']",User's first name,Yes
2,email,users,"['frank@mail.com', 'ivy@mail.com', 'frank@exam...",User's email address,Yes
3,created_at,users,"['2023-10-17', '2022-12-06', '2018-11-28', '20...",Account creation date,Yes
4,product_id,products,"['P0001', 'P0002', 'P0003', 'P0004', 'P0005']",Unique identifier for products,No


In [12]:
df.columns = ['column_name', 'table_name', 'sample_data', 'column_description', 'PII']

In [17]:
df['PII'].unique()

array(['Yes', 'No'], dtype=object)

In [13]:
def get_texts(row):

    sample = str(row['sample_data'])

    return f"""column_name: {row['column_name']} , table_name: {row['table_name']}, 'sample_data': {sample}, 
    'column_description': {row['column_description']}, PII: {row['PII']}"""

In [14]:
df['text'] = df.apply(get_texts, axis=1).tolist()

In [25]:
df['text']

0      column_name: user_id , table_name: users, 'sam...
1      column_name: first_name , table_name: users, '...
2      column_name: email , table_name: users, 'sampl...
3      column_name: created_at , table_name: users, '...
4      column_name: product_id , table_name: products...
                             ...                        
245    column_name: category , table_name: products, ...
246    column_name: order_id , table_name: orders, 's...
247    column_name: user_id , table_name: orders, 'sa...
248    column_name: order_date , table_name: orders, ...
249    column_name: total_amount , table_name: orders...
Name: text, Length: 250, dtype: object

In [19]:
def get_embeddings_openai(texts, model="text-embedding-3-small"):
    embeddings = []

    for text in texts:
        response = openai.Embedding.create(input=text, model=model)
        embeddings.append(response['data'][0]['embedding'])
        
    return np.array(embeddings, dtype='float32')


In [21]:
embeddings = get_embeddings_openai(df['text'])


In [22]:
embeddings[0]

array([ 0.02169088, -0.01320222,  0.07703781, ..., -0.0190675 ,
        0.00260872, -0.02292792], shape=(1536,), dtype=float32)

In [23]:
# Build FAISS index
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [24]:
query = "column_name: book_id, table_name: Books, column_description: unique identifier for books"
query_embed = get_embeddings_openai([query])
D, I = index.search(np.array(query_embed).astype('float32'), k=3)

# Show results
for idx in I[0]:
    print(df.iloc[idx])

column_name                                                  product_id
table_name                                                     products
sample_data               ['P0001', 'P0002', 'P0003', 'P0004', 'P0005']
column_description                       Unique identifier for products
PII                                                                  No
text                  column_name: product_id , table_name: products...
Name: 21, dtype: object
column_name                                                  product_id
table_name                                                     products
sample_data               ['P0001', 'P0002', 'P0003', 'P0004', 'P0005']
column_description                       Unique identifier for products
PII                                                                  No
text                  column_name: product_id , table_name: products...
Name: 55, dtype: object
column_name                                                  product_id
table_name      

In [29]:
query_embed.shape, type(query_embed)

((1, 1536), numpy.ndarray)

In [27]:
I

array([[21, 55, 72]])

In [26]:
query_embed

array([[ 0.03597869, -0.01475553,  0.03335503, ..., -0.01921982,
         0.04177514, -0.02078588]], shape=(1, 1536), dtype=float32)